In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Make NumPy printouts easier to read
np.set_printoptions(precision=3, suppress=True)


# Load the dataset
bike_rentals = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv")
bike_rentals_holdout = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv")


# Define X/Y
bike_rentals["total_rentals"] = bike_rentals.casual + bike_rentals.registered
y = bike_rentals.total_rentals
X = bike_rentals.drop(["total_rentals", "dteday"], axis='columns')

# Set aside the test set immediately
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train.head()

,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c,casual,registered
581,1,12,0,1,3,0.93,22,2.34,-2.9980,7,41
2916,2,10,0,1,2,0.49,22,17.38,18.0032,30,97
4313,3,15,0,0,1,0.53,20,31.48,36.9980,194,182
16075,4,12,0,1,2,0.53,19,7.04,3.9980,24,188
8757,1,17,0,1,2,0.34,13,8.92,7.0010,19,393


In [24]:
# Normalize the data
# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(np.array(X_train))
# print(normalizer.mean.numpy())


scaler = MinMaxScaler()

X_train_normal = scaler.fit_transform(X_train)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(4, activation="relu"),
    tf.keras.layers.Dense(1),
])

# initial garbage prediction
model.predict(X_train_normal)

In [25]:
model.summary()

In [26]:
# Compile using a legacy optimizer cause M1 Mac
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.1),
    loss='mean_squared_error'
)

In [27]:
# Train the neural network!
training_history = model.fit(
    X_train_normal,
    y_train,
    epochs=128,
    validation_split=0.2
)

In [28]:
hist = pd.DataFrame(training_history.history)
hist['epoch'] = training_history.epoch
hist.tail()

def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlim([0, 20])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

plot_loss(training_history)